### Importing Libraries

In [ ]:
import warnings
import tensorflow as tf

In [ ]:
#Viewing Tenserflow version
tf.__version__

In [ ]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization, Dropout

In [ ]:
import gdown

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Downloading Data

#### Downloading data from google drive

In [ ]:
sharedUrl='https://drive.google.com/file/d/1L2-WXzguhUsCArrFUc8EEkXcj33pahoS/view'
dwd_url='https://drive.google.com/uc?export=download&id=' + sharedUrl.split('/')[-2]

id = sharedUrl.split("/")[-2]

output='SVHN_single_grey1.h5'
gdown.download(dwd_url,output,quiet=False)

In [ ]:
 
#df = pd.read_hdf('SVHN_single_grey1.h5') 
#print(df)

In [ ]:
import h5py

data=h5py.File('SVHN_single_grey1.h5','r')

In [ ]:
data.keys()



*   H5PY file has 3 sets of data for train, test and validation

### Extracting data from H5PY file

- Saving data from H5PY file desiered variables

In [ ]:

X_train=data['X_train'][:]
X_test=data['X_test'][:]
X_val=data['X_val'][:]
y_train=data['y_train'][:]
y_test=data['y_test'][:]
y_val=data['y_val'][:]


In [ ]:
'''
X_train=data.get('X_train')
X_test=data.get('X_test')
X_val=data.get('X_val')
y_train=data.get('y_train')
y_test=data.get('y_test')
y_val=data.get('y_val')
'''

### Exploring data

- Data types of downloaded data sets

In [ ]:
print("Training data X-- Shape :", X_train.shape,"and Data Type : ", X_train.dtype)
print("Testing data X-- Shape :", X_test.shape,"and Data Type : ", X_test.dtype)
print("Validation data X-- Shape :", X_val.shape,"and Data Type : ", X_val.dtype)
print("Training data y-- Shape :", y_train.shape,"and Data Type : ", y_train.dtype)
print("Testing data y-- Shape :", y_test.shape,"and Data Type : ", y_test.dtype)
print("Validation data y-- Shape :", y_val.shape,"and Data Type : ", y_val.dtype)

- All data set contains images of size 32 * 32 and their lables
- Train Data:
  - Total Images: 42000
  - Image size: 32 X 32
  - Image/Independent Data type saved as float
  - Dependent or expected output saved as uint8 

- Test Data:
  - Total Images: 18000
  - Image size: 32 X 32
  - Data type saved as float
  - Image/Independent Data type saved as float
  - Dependent or expected output saved as uint8

- Validation Data:
  - Total Images: 60000
  - Image size: 32 X 32
  - Data type saved as float
  - Image/Independent Data type saved as float
  - Dependent or expected output saved as uint8

Show first number in the train data set with expected label

In [ ]:
plt.imshow(X_train[0])    # show first number in the train dataset
plt.show()
print('Label: ', y_train[0])

- on screen we can see there are 3 clear identifiable numbers 1,2,8 but it is tagged to digit 2 only
- current image is taking the output as 2 which is the center value



Randomly taking a value from middle of data set - 1050

In [ ]:
plt.imshow(X_train[1050])    
plt.show()
print('Label: ', y_train[1050])

- we can still notice the similar pattern as it takes the expected value of middle digit

First Value in Test data sets

In [ ]:
plt.imshow(X_test[0])    # show first number in the test dataset
plt.show()
print('Label: ', y_test[0])

In [ ]:
plt.imshow(X_test[200])    
plt.show()
print('Label: ', y_test[200])

First value in Validation data set

In [ ]:
plt.imshow(X_val[0])    # show first number in the val dataset
plt.show()
print('Label: ', y_val[0])

In [ ]:
plt.imshow(X_val[900])   
plt.show()
print('Label: ', y_val[900])

- We can observe the variation in digits in Validation and Test data sets are similar to Train

In [ ]:
#Visualize the data, Plot first 10 images in the triaining set and their labels.
import matplotlib.pyplot as plt
%matplotlib inline
for i in range(10):
    image = X_train[i]
    plt.figure(i)
    plt.imshow(image)
    print('Label: ', y_train[i])

In [ ]:
#view 10 train images
#check for orientation and alignments

fig=plt.figure(figsize=(10,10))

for i in range(1,26):
  img=X_train[i]
  fig.add_subplot(5,5,i)
  plt.imshow(img)

plt.show()
print('Label: ', y_train[1:26])

- We can notice there are multiple image with more than one clear images of digit or just one or a blur image of all
- Orientation is as well not straight for all
- Digits are of different font and sizes as well
- Will try to look the values in grayscale

In [ ]:
#numbers are tilted and not in center
#multiple numbers with similar match
#consider 149 in image or 33 or 11
#check for more orientation and alignments, check in gray scale

fig=plt.figure(figsize=(15,15))

for i in range(1,101):
  img=X_train[i]
  fig.add_subplot(10,10,i)
  plt.imshow(img,cmap='gray')

plt.show()


print('Label: ', y_train[1:101])

### Reshape and normalize the train and test features 

### Data Pre Processing

In [ ]:
#Re importing to set the data from this point -- comment the section later

X_train=data['X_train'][:]
X_test=data['X_test'][:]
X_val=data['X_val'][:]
y_train=data['y_train'][:]
y_test=data['y_test'][:]
y_val=data['y_val'][:]


In [ ]:
X_train.shape

- Current shape of data is - 42,000 images with 32X32 image size
- We have to reshape the data before model creation or we can do it while model
 creation as well with model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))
 

#### Reshape train and test sets into compatible shapes

- Sequential model in tensorflow.keras expects data to be in the format (n_e, n_h, n_w, n_c) 
n_e= number of examples, n_h = height, n_w = width, n_c = number of channels
- labels will be converted to categorical variables

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 32,32,1))
X_test = X_test.reshape((X_test.shape[0], 32,32,1))
X_val = X_val.reshape((X_val.shape[0],32,32,1))

#### Normalizing data
- dividing the RGB codes with 255 (which is the maximum RGB code minus the minimum RGB code)
- normalize X_train, X_test & X_val
- making sure values are float so that we can get decimal points after division

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

X_train /= 255
X_test /= 255
X_val /= 255

Print shape of data and number of images

In [ ]:
print("X_train shape:", X_train.shape)
print("Images in X_train:", X_train.shape[0])
print("Images in X_test:", X_test.shape[0])
print("Images in X_val:", X_val.shape[0])
print("Max value in X_train:", X_train.max())
print("Min value in X_train:", X_train.min())
print("Max value in X_test:", X_test.max())
print("Min value in X_test:", X_test.min())
print("Max value in X_val:", X_val.max())
print("Min value in X_val:", X_val.min())

#### One hot encode the labels for train and test data

##### One-hot encode the class vector

- convert class vectors (integers) to binary class matrix
- convert y_train, y_test & y_val
- number of classes: 10

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
y_val = to_categorical(y_val,num_classes=10)

print("Shape of y_train:", y_train.shape)
print("One value of y_train:", y_train[0])

print("Shape of y_test:", y_test.shape)
print("One value of y_test:", y_test[0])

print("Shape of y_val:", y_val.shape)
print("One value of y_val:", y_val[0])

### Define the model architecture using TensorFlow with a flatten layer followed by dense layers with activation as ReLu and softmax

- Initialize a sequential model
- Create a sequential model
- flatten the data
  - add Flatten later
  - flatten layers flatten 2D arrays to 1D array before building the fully connected layers
- add 'n' dense layers
  - number of neurons in first layer: to be experimented
  - number of neurons in last layer: number of classes - 10
  - activation function in first layer: relu
  - activation function in last layer: softmax

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

model = Sequential()
#model.add(Flatten(input_shape=(32, 32, 1)))#reshapes to (1024)=32*32*1#we can use this explicitly as well but keras can analyze the input itself and provide the result
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation="softmax"))

Compile and fit the model
  - let's compile our model
    - loss: "categorical_crossentropy"
    - metrics: "accuracy"
    - optimizer: "adam"

  - then next step will be to fit model
    - give train data - training features and labels
    - batch size: Test
    - epochs: 10 - forevaluation for accuracy we have to give higher
    - give validation data - testing features and labels

In [ ]:
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=30, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- with just basic setup and 2 dense layers we are getting low accuracy which is less than tossing a coin
- Accuracy didnt chagned or converged during 10 accuracies and stayed at same level
- will try to change the learning rate and see if it help increase the accuracy

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation="softmax"))

#updating learning rate
adam = optimizers.Adam(lr = 0.01)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=30, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- With just updating the learing rate is not helping the accuracy at all
- we might have to add more layers and use other features to improve the accuracy

Updated sequential Model:

- Initialize
- Flatten
- Batch Normalization
- Activation Relu
- dense layers -5
- Activation Softmax for Final layer


In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(10))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr = 0.01)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=30, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- We can observe the accuracy is drastically increased with introduction of batch normalization
- We can try with updated learning rate
  - Current learning rate is not convergin as fast as we expected, we can increase it to capture the behaviour

In [ ]:
#Learning Rate updated from 0.01 to 0.09
model = Sequential()
model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(10))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr = 0.09)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- Accuracy is improved after increasing the learning rate from 0.01 to 0.09
- Our current model is not able to capture the complex relations with image and label, we can improve more by increasing the layers to capture the complex relations

- We are adding more layers and reducing the learning rate

In [ ]:
# Adding multiple dense layers
model = Sequential()
model.add(Flatten())
model.add(Dense(1024))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(512))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(256))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(128))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(10))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr = 0.009)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=30, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- Accuracy is improved for Train and Test both
- Will try to update the modle with kernel initialization

- Updating model with kernel_initializer='he_normal'

In [ ]:
#kernel_initializer='he_normal'

model = Sequential()
model.add(Flatten())
model.add(Dense(1024,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(256,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(128,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(10,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr = 0.009)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=30, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- Not much impact on accuracy after introducing kernel initialization for test accuracy

- Updating Adam Optimizer with decay

In [ ]:
#kernel_initializer='he_normal'

model = Sequential()
model.add(Flatten())
model.add(Dense(1024,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(256,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(128,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(10,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr=0.009, decay=1e-6)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=30, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- Train and Test accuracies are close by after 10 epochs
- At this point we can try SGD optimzer as well to capture its behaviour

In [ ]:
#SGD Optimizer
model2 = Sequential()
model2.add(Flatten())
model2.add(Dense(1024,kernel_initializer='he_normal'))
model2.add(BatchNormalization())                    
model2.add(Activation('relu'))    
model2.add(Dense(512,kernel_initializer='he_normal'))
model2.add(BatchNormalization())                    
model2.add(Activation('relu'))    
model2.add(Dense(256,kernel_initializer='he_normal'))
model2.add(BatchNormalization())                    
model2.add(Activation('relu'))    
model2.add(Dense(128,kernel_initializer='he_normal'))
model2.add(BatchNormalization())                    
model2.add(Activation('relu'))    
model2.add(Dense(10,kernel_initializer='he_normal'))
model2.add(Activation('softmax'))

#updating learning rate
adam = optimizers.SGD(lr=0.009, decay=1e-6)
# Compile the model
model2.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="sgd")

# Fit the model
model2.fit(x=X_train, y=y_train, batch_size=30, epochs=10, validation_data=(X_test, y_test))

In [ ]:
results = model2.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- train and test accuracies are more comparable and close for adam optimizer, will proceed with that only


-  Increase the Epochs

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(1024,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(512,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(256,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(128,kernel_initializer='he_normal'))
model.add(BatchNormalization())                    
model.add(Activation('relu'))    
model.add(Dense(10,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

#updating learning rate
adam = optimizers.Adam(lr=0.01, decay=1e-6)
# Compile the model
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# Fit the model
model.fit(x=X_train, y=y_train, batch_size=30, epochs=150, validation_data=(X_test, y_test))

In [ ]:
results = model.evaluate(X_test, y_test)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

In [ ]:
results = model.evaluate(X_val, y_val)
print('Accuracy: %f ' % (results[1]*100))
print('Loss: %f' % results[0])

- We can observe after giving another 100 epochs to the system we are getting train accuracy more than 95%, test accuracy more than 80% and Validation accuracy close to 95%

- Comapre the Test and Validation image with expected and predicted label

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc


In [ ]:
Y_pred_val_cls = (model.predict(X_val) > 0.5).astype("int32")

In [ ]:
import matplotlib.pyplot as plt

X_val_temp=data['X_val'][:]
y_val_temp=data['y_val'][:]

plt.imshow(X_val_temp[0])    # show first number in the val dataset
plt.show()

print('Label: ', y_val_temp[0])
print('Actual Label - one hot encoded: ', y_val[0])
print('Predicted Label - one hot encoded: ', Y_pred_val_cls[0])

In [ ]:
plt.imshow(X_val_temp[900])    
plt.show()

print('Label: ', y_val_temp[900])
print('Actual Label - one hot encoded:  ', y_val[900])
print('Predicted Label - one hot encoded: ', Y_pred_val_cls[900])

In [ ]:
plt.imshow(X_val_temp[9500])    
plt.show()

print('Label: ', y_val_temp[9500])
print('Actual Label - one hot encoded: ', y_val[9500])
print('Predicted Label - one hot encoded: ', Y_pred_val_cls[9500])

### Model Review

1.   Base Model:
  - with flatten & 2 dense layer having relu and softmax as last layer
  - adam optimizer
  - noticed the accuracy is very low
  - expected reasons could be due to high complex relations with image and its expected label which the simple model is not able to capture and predict
2.   Base Model + updated learning Rate for optimizer:
  - with updated learning rate accuracies were improved but very slightly
  - even with slightly higher learning rate the convergance is not happening and accuracy remained low
3.   Base Model + Batch Normalization:
  - introducing batch normalization we noticed significant increase in the accuracy
  - we are runnig all the model with same epochs and batch to get similar comparison
4.   Base Model + Batch Normalization + Additional Dense Layers
  - additonal layers gave us much better accuracies
5.   Base Model + Batch Normalization + Additional Dense Layers + Kernel initializer
  - kernel initializer couldnt improve much on accuracy
6.   Base Model + Batch Normalization + Additional Dense Layers + SGD Optimizer
  - Accuracies were similar or worse
7.   Base Model with Adam Otimizer having 150 epochs
  - we were able to get very high train and test accuracies as well as accuracies for Val data set as well